In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import os
import re
import numpy as np
import pandas as pd
import collections

In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import random

#physical_devices = tf.config.list_physical_devices('GPU') 
#tf.config.experimental.set_memory_growth(physical_devices[0], True)
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

seed_value= 0
os.environ['PYTHONHASHSEED']=str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

In [ ]:
import sys
ipython = get_ipython()

def hide_traceback(exc_tuple=None, filename=None, tb_offset=None,
                      exception_only=False, running_compiled_code=False):
       etype, value, tb = sys.exc_info()
       return ipython._showtraceback(etype, value, ipython.InteractiveTB.get_exception_only(etype, value))

ipython.showtraceback = hide_traceback

In [ ]:
x_name = os.path.join(os.getcwd(), 'raw', 'SCDC_Track1_2차', f"{'cst_feat_jan'}.csv")
df_x = pd.read_csv(x_name, index_col=0)

y_name = os.path.join(os.getcwd(), 'raw', 'SCDC_Track1_2차', f"{'train'}.csv")
df_y = pd.read_csv(y_name, index_col=0)

v_name = os.path.join(os.getcwd(), 'raw', 'SCDC_Track1_2차', f"{'val'}.csv")
df_v = pd.read_csv(v_name, index_col=0)

df_yv = pd.concat([df_y, df_v])
df = pd.merge(df_x, df_yv, on='cst_id_di')

In [ ]:
X = df.drop(columns = ['MRC_ID_DI'], axis=1)
y = df['MRC_ID_DI']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, random_state=1, stratify = y)

In [ ]:
train = X_train
train['cst_id_di'] = y_train.index
train = train.set_index('cst_id_di')
train['MRC_ID_DI'] = y_train

train_0 = train[train['MRC_ID_DI'] == 0].sample(frac=1)
train_1 = train[train['MRC_ID_DI'] == 1].sample(frac=1)
train_2 = train[train['MRC_ID_DI'] == 2].sample(frac=1)
train_3 = train[train['MRC_ID_DI'] == 3].sample(frac=1)
train_4 = train[train['MRC_ID_DI'] == 4].sample(frac=1)
train_5 = train[train['MRC_ID_DI'] == 5].sample(frac=1)
train_6 = train[train['MRC_ID_DI'] == 6].sample(frac=1)
train_7 = train[train['MRC_ID_DI'] == 7].sample(frac=1)
train_8 = train[train['MRC_ID_DI'] == 8].sample(frac=1)
train_9 = train[train['MRC_ID_DI'] == 9].sample(frac=1)
train_10 = train[train['MRC_ID_DI'] == 10].sample(frac=1)

sample_size = min(len(train_0),len(train_1), len(train_2), len(train_3), len(train_4), len(train_5), len(train_6), len(train_7),
                 len(train_8), len(train_9), len(train_10))

train_f = pd.concat([train_0.head(sample_size), train_1.head(sample_size), train_2.head(sample_size), train_3.head(sample_size),
                    train_4.head(sample_size), train_5.head(sample_size), train_6.head(sample_size), train_7.head(sample_size), 
                    train_8.head(sample_size), train_9.head(sample_size), train_10.head(sample_size)]).sample(frac=1)

X_train = train_f.drop(columns = ['MRC_ID_DI'], axis=1)
y_train = train_f['MRC_ID_DI']

In [ ]:
import sklearn.feature_selection

select = sklearn.feature_selection.SelectKBest(k=45)
selected_features = select.fit(X_train, y_train)
indices_selected = selected_features.get_support(indices=True)
colnames_selected = [X_train.columns[i] for i in indices_selected]

X_train = X_train[colnames_selected]

In [ ]:
from itertools import combinations
from sklearn.preprocessing import PolynomialFeatures

X_train = X_train.astype(np.float16)
combos = list(combinations(list(X_train.columns), 2))
colnames = list(X_train.columns) + ['_'.join(x) for x in combos]

poly = PolynomialFeatures(interaction_only=True, include_bias=False)
X_train = poly.fit_transform(X_train)
X_train = pd.DataFrame(X_train)
X_train.columns = colnames


noint_indicies = [i for i, x in enumerate(list((X_train == 0).all())) if x]
X_train = X_train.drop(X_train.columns[noint_indicies], axis = 1)

In [ ]:
train = X_train
train['cst_id_di'] = y_train.index
train = train.set_index('cst_id_di')
train['MRC_ID_DI'] = y_train

X_train = train.drop(columns = ['MRC_ID_DI'], axis=1)
y_train = train['MRC_ID_DI']

In [ ]:
import sklearn.feature_selection

select = sklearn.feature_selection.SelectKBest(k=331)
selected_features = select.fit(X_train, y_train)
indices_selected = selected_features.get_support(indices=True)
colnames_selected = [X_train.columns[i] for i in indices_selected]

X_train = X_train[colnames_selected]

In [ ]:
from itertools import combinations
from sklearn.preprocessing import PolynomialFeatures

X_test = X_test.astype(np.float16)
combos = list(combinations(list(X_test.columns), 2))
colnames = list(X_test.columns) + ['_'.join(x) for x in combos]

poly = PolynomialFeatures(interaction_only=True, include_bias=False)
X_test = poly.fit_transform(X_test)
X_test = pd.DataFrame(X_test)
X_test.columns = colnames


noint_indicies = [i for i, x in enumerate(list((X_test == 0).all())) if x]
X_test = X_test.drop(X_test.columns[noint_indicies], axis = 1)

In [ ]:
test = X_test
test['cst_id_di'] = y_test.index
test = test.set_index('cst_id_di')
test['MRC_ID_DI'] = y_test

X_test = test.drop(columns = ['MRC_ID_DI'], axis=1)
y_test = test['MRC_ID_DI']

In [ ]:
X_test = X_test[colnames_selected]

In [ ]:
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

In [ ]:
input_ = tf.keras.Input(dtype = tf.float32, shape = (len(X_train.columns),))

dense_layer_1_1 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(input_)
dense_layer_1_2 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(dense_layer_1_1)
dense_layer_1_3 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(dense_layer_1_2)
dense_layer_1_4 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(dense_layer_1_3)

dropout_1_5 = tf.keras.layers.Dropout(rate = 0.2)(dense_layer_1_1)

output = tf.keras.layers.Dense(units = 11, activation = tf.nn.softmax)(dropout_1_5)

In [ ]:
model = tf.keras.Model(inputs=input_, outputs=output)

In [ ]:
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), 
              optimizer=tf.compat.v1.train.AdamOptimizer(learning_rate=0.00005), 
              metrics=['accuracy'])
print(model.summary())

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=40)
history = model.fit(x=X_train, y=y_train, batch_size=32, epochs=10000, verbose=1, validation_split=0.1, callbacks=[callback])

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])

plt.title('model accuracy')
plt.ylabel('acc')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()

In [ ]:
score = model.evaluate(x=X_test, y=y_test, verbose=1)

print("Test Score:", score[0])
print("Test ACC:", score[1])

In [ ]:
preds = model.predict(X_test, verbose=1)

In [ ]:
def LIFT(preds, y_test, cls): # >=2.5

    n_score = preds[:, cls]
    ind = np.argsort(-n_score)[:len(n_score)//5] #예측 score 상위 20%
    
    y_20 = y_test[ind]
    y_20_flat = y_20[:, cls]
    y_20_final = np.count_nonzero(y_20_flat == 1)
    
    y_test_flat = np.argmax(y_test, axis=1)
    y_test_final = y_test_flat[y_test_flat == cls] 
    lift = (y_20_final/len(y_20))/ (len(y_test_final)/len(y_test))
    print('LIFT Accuracy: ',  lift)
    return lift

In [ ]:
lift_score = [0, 0, 0]
lift_score[0] = LIFT(preds, y_test, 3)
lift_score[1] = LIFT(preds, y_test, 4)
lift_score[2] = LIFT(preds, y_test, 7)
avg_lift = sum(lift_score) / 3

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

fpr = dict()
tpr = dict()
roc_auc = dict()
for i in [3, 4, 7]:
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], preds[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
avg_auroc = sum([roc_auc[3], roc_auc[4], roc_auc[7]]) / 3

In [ ]:
final_score = (avg_lift)*0.7 + (avg_auroc)*0.3
print(avg_lift, avg_auroc, final_score)

In [ ]:
#331- 0.2381